# Project: Investigate Imdb Movie Dataset 

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction
IMDB Movie Data:
This dataset contains information about 10,000 movies collected from The Movie
Database (TMDb),including user ratings and revenue.
Data provided by [Udacity](https://d17h27t6h515a5.cloudfront.net/topher/2017/October/59dd1c4c_tmdb-movies/tmdb-movies.csv)<br>
Original data provided by [Kaggle](https://www.kaggle.com/tmdb/tmdb-movie-metadata)<br>

**Dataset features:**<br>
- id,
- imdb_id,
- popularity,
- budget,
- revenue,
- original_title,
- cast,
- homepage,
- director,
- tagline,
- keywords,
- overview,
- runtime,
- genres,
- production_companies,
- release_date,
- vote_count,
- vote_average,
- release_year,
- budget_adj*,
- revenue_adj*<br><br>
**The final two columns ending with “_adj” show the budget and revenue of the associated movie in terms of 2010 dollars, accounting for inflation over time.*

Not all features are relevant to the questions this analysis is to answer. Therefore, a few will be taken out.

### Questions:
>describe the questions that you plan on exploring over the course of the report. Try to build your report around the analysis of at least one dependent variable and three independent variables.
>


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
%matplotlib inline

<a id='wrangling'></a>
## Data Wrangling
In this section data is <a href = '#load'>loaded</a>, <a href = #check>checked for cleanliness</a>, and then <a href =#trim>trimed and cleaned</a> for analysis. 

<a id = 'load'></a>

<a id = 'load'></a>

In [2]:
df_raw_data = pd.read_csv('../../Datasets/movies.csv')
df_raw_data.head(3)

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/15,5562,6.5,2015,1.379999e+08,1.392446e+09
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/15,6185,7.1,2015,1.379999e+08,3.481613e+08
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/15,2480,6.3,2015,1.012000e+08,2.716190e+08


In [3]:
print('Dataset consist of {} rows and {} columns.'.format(df_raw_data.shape[0],df_raw_data.shape[1]))

Dataset consist of 10866 rows and 21 columns.


<a id = 'check'></a>
#### Number of Duplicates

In [4]:
print('Of {} rows, {} are unique and {} is a duplicate.'.format(df_raw_data.shape[0],df_raw_data.id.nunique(),df_raw_data.duplicated().sum()))

Of 10866 rows, 10865 are unique and 1 is a duplicate.


#### Number of Missing Values for Each Column

In [5]:
pd.DataFrame(df_raw_data.isna().sum()).transpose()

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,0,10,0,0,0,0,76,7930,44,2824,...,4,0,23,1030,0,0,0,0,0,0


#### Number of Empty Rows

In [6]:
df_raw_data.isna().all(1).sum()

0

#### Number of 0s for Each Column

In [7]:
pd.DataFrame(df_raw_data.eq(0).sum()).transpose()

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,0,0,0,5696,6016,0,0,0,0,0,...,0,31,0,0,0,0,0,0,5696,6016


#### Number of Unique Values for Each Column
Although this information is not of importance for every column, it helps us ensure not more than one entry is made fore one movie. 

In [8]:
pd.DataFrame(df_raw_data.nunique()).transpose()

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,10865,10855,10814,557,4702,10571,10719,2896,5067,7997,...,10847,247,2039,7445,5909,1289,72,56,2614,4840


In [9]:
df_raw_data[['original_title','director','cast']].duplicated().sum()


2

>At first glance, number of unique value for ```original_title``` suggests that we have duplicate entries. However, running the code above explains that there are different movies with the same name.Moreover, it suggests that we have one more duplicate row that perhaps was not detected by ```duplicated()``` earlier du to a difference in one of the columns' entry.

#### Check Columns' Datatype

In [10]:
df_raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10866 entries, 0 to 10865
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    10866 non-null  int64  
 1   imdb_id               10856 non-null  object 
 2   popularity            10866 non-null  float64
 3   budget                10866 non-null  int64  
 4   revenue               10866 non-null  int64  
 5   original_title        10866 non-null  object 
 6   cast                  10790 non-null  object 
 7   homepage              2936 non-null   object 
 8   director              10822 non-null  object 
 9   tagline               8042 non-null   object 
 10  keywords              9373 non-null   object 
 11  overview              10862 non-null  object 
 12  runtime               10866 non-null  int64  
 13  genres                10843 non-null  object 
 14  production_companies  9836 non-null   object 
 15  release_date       

>Datatype for ```release_date``` should change to ```datetime```. 

<a id ='trim'></a>	
### Data Cleaning and Triming

**Drop irrelevent columns**

In [11]:
df_raw_data.drop(['imdb_id','popularity','homepage','tagline','overview','budget','revenue'],axis =1, inplace = True)

**Delete Duplicate Rows**

In [12]:
df_raw_data.drop_duplicates(inplace = True)
print('Number of duplicate rows: {}'.format(df_raw_data.duplicated().sum()))

Number of duplicate rows: 0


In [13]:
#Drop rows with entries on the same film
df_raw_data[df_raw_data[['original_title','director','cast']].duplicated()]

,id,original_title,cast,director,keywords,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
6701,16781,Madea's Family Reunion,Tyler Perry|Blair Underwood|Lynn Whitfield|Bor...,Tyler Perry,spanking|based on play,110,Drama|Comedy|Romance,Lions Gate Films,2/24/06,63,6.0,2006,6.490015e+06,6.190557e+07


In [14]:
df_raw_data[df_raw_data['original_title'] == "Madea's Family Reunion"]

,id,original_title,cast,director,keywords,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
4063,28004,Madea's Family Reunion,Tyler Perry|Blair Underwood|Lynn Whitfield|Bor...,Tyler Perry,NaN,0,Comedy,NaN,1/25/02,49,5.9,2002,7.273568e+06,0.000000e+00
6701,16781,Madea's Family Reunion,Tyler Perry|Blair Underwood|Lynn Whitfield|Bor...,Tyler Perry,spanking|based on play,110,Drama|Comedy|Romance,Lions Gate Films,2/24/06,63,6.0,2006,6.490015e+06,6.190557e+07


In [15]:
df_raw_data.drop([4063],axis = 0, inplace = True)
df_raw_data[df_raw_data['original_title'] == "Madea's Family Reunion"]

,id,original_title,cast,director,keywords,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
6701,16781,Madea's Family Reunion,Tyler Perry|Blair Underwood|Lynn Whitfield|Bor...,Tyler Perry,spanking|based on play,110,Drama|Comedy|Romance,Lions Gate Films,2/24/06,63,6.0,2006,6.490015e+06,6.190557e+07


#### Fix Datatypes

**Column ```release_date```**<br>
Datatype for release date is string. It should be converted to datetime.

In [16]:
df_raw_data.release_date = df_raw_data.release_date.apply(lambda d: d.replace(d,d[:-2]))+ df_raw_data.release_year.astype(str)
df_raw_data.release_date = pd.to_datetime(df_raw_data.release_date)
df_raw_data.release_date

0       2015-06-09
1       2015-05-13
2       2015-03-18
3       2015-12-15
4       2015-04-01
           ...    
10861   1966-06-15
10862   1966-12-21
10863   1966-01-01
10864   1966-11-02
10865   1966-11-15
Name: release_date, Length: 10864, dtype: datetime64[ns]

> **Explanation**: At first attempt in converting the datatype, the results achieved where not as desired. dates such as d/m/66 were converted to 2066-m-d. So the piece of code below was used to ensure the date in column ```release_date```was more readable:<br>
``` df_raw_data.release_date.apply(lambda d: d.replace(d,d```[ : -2]```))+ df_raw_data.release_year.astype(str)``` 
<br>Conversion took place once this problem was fixed. Consequently, given that column ```release_year```is no longer of use, it is dropped.

**Column ``` id```**<br>
Datatype for id is ```in```. But it has no numerical meaning. It should be converted to string.

### Trim data entries

**Remove white space**

In [17]:
for col in df_raw_data.columns:
    if df_raw_data[col].dtype == str:
        df_raw_data[col] = df_raw_data[col].str.strip().copy()

**Fix columns with multipl-value data entries**

#### Columns ```cast```,  ```keywords```, ```genres```, ```production_companies```
A number of columns have more than one entry for each record. For example column ```cast```contains names of several actors separated by ```'|'```.
This will make it difficult to efficiently analyse the data based on one specific entry. Let's say, we want to have a range of info on movies that have actor 'X' featuring. Current structure of data does not accomodate that easily. Therefore, the entries should be splitted. 

In [18]:
df_raw_data[['cast','keywords','genres','production_companies']].head(3)

,cast,keywords,genres,production_companies
0,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,monster|dna|tyrannosaurus rex|velociraptor|island,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...
1,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,future|chase|post-apocalyptic|dystopia|australia,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...
2,Shailene Woodley|Theo James|Kate Winslet|Ansel...,based on novel|revolution|dystopia|sequel|dyst...,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...


> *Given the size of dataset, to make the process faster the dataset is to be devided into a number of subd-atasets(tables) which later, depending on the requirements of the analysis,will be merged. Consequently, handling null and 0 values will be more pragmatic this way

**Create movie_dataset**

In [19]:
df_movies = df_raw_data[['id', 'original_title','release_date','release_year','runtime','vote_average','budget_adj','revenue_adj']].copy()
print('Number of Nan entries : {}\nNumber of 0 entries: {} to be found in column: {}'.format(df_movies.isna().sum().sum(),df_movies.eq(0).sum().sum(),list(df_movies.columns[df_movies.eq(0).any() == True])[0]))

Number of Nan entries : 0
Number of 0 entries: 11741 to be found in column: runtime


>In order to fix this, 0 values will be replace by the average runtime of each genre. Therefore, this will be fixed once the data entries for columns ```genres``` are fixed.

In [20]:
df_movies.to_csv('../../Datasets/movies_dataset.csv', index = False)

**Create actors_dataset**

In [21]:
df_actors = df_raw_data[['id','cast']].copy()
print('Number of Nan entries : {}\nNumber of 0 entries: {} '.format(df_actors.isna().sum().sum(),df_actors.eq(0).sum().sum()))

Number of Nan entries : 76
Number of 0 entries: 0 


In [22]:
df_actors.dropna(axis = 0, inplace = True)
df_actors.rename(columns = {'cast':'actor'}, inplace = True)
df_actors.to_csv('../../Datasets/df_actors.csv', index = False)

>column's name changed from ```cast ```to ```actor```

In [23]:
df_actors.head(2)

,id,actor
0,135397,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...
1,76341,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...


**Create genres_dataset**

In [24]:
df_genres = df_raw_data[['id','genres']].copy()
df_genres.rename(columns = {'genres':'genre'}, inplace = True)
print('Number of Nan entries : {}\nNumber of 0 entries: {} '.format(df_genres.isna().sum().sum(),df_genres.eq(0).sum().sum()))

Number of Nan entries : 23
Number of 0 entries: 0 


In [54]:
df_genres.dropna(axis = 0, inplace = True)
df_genres.to_csv('../../Datasets/genres_dataset.csv', index = False)

>column's name changed from ```genres ```to ```genre```

In [55]:
df_genres.query('id == "127717"')

,id,genre


In [26]:
df_genres.head(2)

,id,genre
0,135397,Action|Adventure|Science Fiction|Thriller
1,76341,Action|Adventure|Science Fiction|Thriller


**Create keywords_dataset**

In [27]:
df_keywords = df_raw_data[['id','keywords']].copy()
print('Number of Nan entries : {}\nNumber of 0 entries: {} '.format(df_keywords.isna().sum().sum(),df_keywords.eq(0).sum().sum()))

Number of Nan entries : 1492
Number of 0 entries: 0 


In [28]:
df_keywords.dropna(axis = 0, inplace = True)
df_keywords.rename(columns = {'keywords':'keyword'}, inplace = True)
df_keywords.to_csv('../../Datasets/df_keywords.csv', index = False)

>column's name changed from ```keywords ```to ```keyword```

In [29]:
df_keywords.head(2)

,id,keyword
0,135397,monster|dna|tyrannosaurus rex|velociraptor|island
1,76341,future|chase|post-apocalyptic|dystopia|australia


**Create production_companies_dataset**

In [30]:
df_production_companies = df_raw_data[['id','production_companies']].copy()
print('Number of Nan entries : {}\nNumber of 0 entries: {} '.format(df_production_companies.isna().sum().sum(),df_production_companies.eq(0).sum().sum()))

Number of Nan entries : 1029
Number of 0 entries: 0 


In [31]:
df_production_companies.dropna(axis = 0, inplace = True)
df_production_companies.rename(columns = {'production_companies':'production_company'}, inplace = True)
df_production_companies.to_csv('../../Datasets/df_production_companies.csv', index = False)

>column's name changed from ```production_companies ```to ```production_company```

In [32]:
df_production_companies.head(2)

,id,production_company
0,135397,Universal Studios|Amblin Entertainment|Legenda...
1,76341,Village Roadshow Pictures|Kennedy Miller Produ...


**Function: fix multiple-value data entries**

In [33]:
def split_data (input_dataframe, column_to_trim):
    df_to_trim = input_dataframe[input_dataframe[column_to_trim].str.find('|') != -1].copy()
    df_temp = input_dataframe.copy()
    ind = df_to_trim.index
    for i in ind:
        record_to_fix = df_temp.loc[i].copy()
        df_temp.drop([i], axis = 0, inplace = True)
        temp = record_to_fix.copy()
        for j in range(record_to_fix[column_to_trim].count('|')+1):
            temp[column_to_trim] = record_to_fix[column_to_trim].split('|')[j]
            df_temp = df_temp.append(temp)
    return df_temp

**Trim data**

In [34]:
df_actors_trimmed = split_data(df_actors,'actor')
df_actors_trimmed.to_csv('../../Datasets/actors_dataset_trimmed.csv', index = False)
df_genres_trimmed = split_data(df_genres, 'genre')
df_genres_trimmed.to_csv('../../Datasets/genres_dataset_trimmed.csv', index = False)
df_production_co_trimmed = split_data(df_production_companies, 'production_company')
df_production_co_trimmed.to_csv('../../Datasets/production_companies_dataset_trimmed.csv', index = False)
df_keywords_trimmed = split_data(df_keywords, 'keyword')
df_keywords_trimmed.to_csv('../../Datasets/keywords_dataset_trimmed.csv', index = False)

In [35]:
df_actors = pd.read_csv('../../Datasets/actors_dataset_trimmed.csv')
df_genres = pd.read_csv('../../Datasets/genres_dataset_trimmed.csv')
df_production_companies = pd.read_csv('../../Datasets/production_companies_dataset_trimmed.csv')
df_keywords= pd.read_csv('../../Datasets/keywords_dataset_trimmed.csv')

In [36]:
df_actors[(df_actors.id==135397) | (df_actors.id==76341)]

,id,actor
144,135397,Chris Pratt
145,135397,Bryce Dallas Howard
146,135397,Irrfan Khan
147,135397,Vincent D'Onofrio
148,135397,Nick Robinson
149,76341,Tom Hardy
150,76341,Charlize Theron
151,76341,Hugh Keays-Byrne
152,76341,Nicholas Hoult
153,76341,Josh Helman


In [37]:
df_genres[(df_genres.id==135397) | (df_genres.id==76341)]

,id,genre
2327,135397,Action
2328,135397,Adventure
2329,135397,Science Fiction
2330,135397,Thriller
2331,76341,Action
2332,76341,Adventure
2333,76341,Science Fiction
2334,76341,Thriller


In [38]:
df_production_companies[(df_production_companies.id==135397) | (df_production_companies.id==76341)]

,id,production_company
3440,135397,Universal Studios
3441,135397,Amblin Entertainment
3442,135397,Legendary Pictures
3443,135397,Fuji Television Network
3444,135397,Dentsu
3445,76341,Village Roadshow Pictures
3446,76341,Kennedy Miller Productions


<a id = 'after_cleaning'></a>

In [39]:
df_keywords[(df_keywords.id==135397) | (df_keywords.id==76341)]

,id,keyword
931,135397,monster
932,135397,dna
933,135397,tyrannosaurus rex
934,135397,velociraptor
935,135397,island
936,76341,future
937,76341,chase
938,76341,post-apocalyptic
939,76341,dystopia
940,76341,australia


**Fix 0 values**<br>
Columns ```runtime```, ```budget_adj```and ```revenue_adj```have considerable number of 0 values that will impact the analysis. 0s will be replace by the average value of each column for each genre.

**Fix 0 values for ```runtime```**

In [ ]:
def fix_zero_values(column_name):
    df_movies_temp2 = df_movies[['id',column_name]].copy()
    #genre is saved in a different dataset. merge datasets first
    df_to_fix = pd.merge(df_genres, df_movies_temp2,how = 'inner',left_on = 'id', right_on = 'id')
    #take out zeros so the average will not be affected
    df_to_fix_Nonzero = df_to_fix[df_to_fix[column_name] != 0].copy()
    df_to_fix_zero = df_to_fix[df_to_fix[column_name] == 0].copy()
    #calculate average value of the column for each genre
    average_column_value = (df_to_fix_Nonzero.groupby('genre').mean()[column_name])

    #replace zero values
    for i in df_to_fix_zero.index:
        genre = df_to_fix_zero.loc[i,'genre']
        average_value = average_column_value[genre]
        df_to_fix_zero.loc[i,column_name] = average_value
   
    df_zero_value_fixed = df_to_fix_zero.copy()

    #update values of column_name in movies dataset
    df_movies_index = df_movies[df_movies[column_name] == 0].index
    for i in df_movies_index:
        movie_id = df_movies.loc[i,'id']
        if movie_id in df_zero_value_fixed.id.unique():
            column_value = df_zero_value_fixed[df_zero_value_fixed.id == movie_id][column_name].mean()
            df_movies.loc[i,column_name] = column_value

In [95]:
fix_zero_values('runtime')
fix_zero_values('revenue_adj')
fix_zero_values('budget_adj')
print('Number of zero values in runtime columns:',df_movies.runtime.eq(0).sum())
print('Number of zero values in revenue_adj columns:',df_movies.revenue_adj.eq(0).sum())
print('Number of zero values in budget_adj columns:',df_movies.budget_adj.eq(0).sum())

Number of zero values in runtime columns: 0
Number of zero values in revenue_adj columns: 0
Number of zero values in budget_adj columns: 0


> The rows left are the ones for which value of genre column is Nan. calculation of budget/revenue without any clue will not be pragmatic. Therefore, they will be dropped

In [96]:
revenue_adj_id_list = df_movies.query('revenue_adj == "0"').index
df_movies.drop(revenue_adj_id_list, axis = 0, inplace = True)
print('Number of zero values in revenue_adj columns:',df_movies.revenue_adj.eq(0).sum())
print('Number of zero values in budget_adj columns:',df_movies.budget_adj.eq(0).sum())

Number of zero values in revenue_adj columns: 0
Number of zero values in budget_adj columns: 0
